In [1]:
import pandas as pd
import numpy as np
from HFC125_hmimCl_PR import configuration
from pyomo.environ import (ConcreteModel,
                           SolverFactory,
                           units as pyunits)

import idaes.logger as idaeslog
import pyomo.contrib.parmest.parmest as parmest

import pytest

from idaes.core import FlowsheetBlock
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)

from idaes.core.util.model_statistics import degrees_of_freedom

In [2]:
def PR_model(data):
    """
    Function which returns initilized model.

    Args:
        data: pandas DataFrame with data

    Returns:
        initialized model
    """
    eps = 0.1
    init_kappa_2_1A = -0.05
    init_kappa_1_2A = -0.05
    m = ConcreteModel()

    m.fs = FlowsheetBlock(default={"dynamic": False})

    m.fs.properties = GenericParameterBlock(default=configuration)

    m.fs.state_block = m.fs.properties.state_block_class(
        default={"parameters": m.fs.properties,
                 "defined_state": True})
    x = float(data['x_R125'])+eps
    m.fs.state_block.flow_mol.fix(1)
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.pressure.fix(float(data["pressure"]))
    m.fs.state_block.mole_frac_comp['hmimCl'].fix(1-x)
    m.fs.state_block.mole_frac_comp['R125'].fix(x)

    # parameter - kappa_A_ij (set at 0.3, 0 if i=j)
    m.fs.properties.PR_kappa_A['hmimCl','hmimCl'].fix(0)
    m.fs.properties.PR_kappa_A['hmimCl', 'R125'].fix(init_kappa_2_1A)
    m.fs.properties.PR_kappa_A['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_A['R125', 'hmimCl'].fix(init_kappa_1_2A)
    m.fs.properties.PR_kappa_B['hmimCl', 'hmimCl'].fix(0)
    m.fs.properties.PR_kappa_B['hmimCl', 'R125'].fix(0)
    m.fs.properties.PR_kappa_B['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_B['R125', 'hmimCl'].fix(0)
    m.fs.properties.PR_kappa_C['hmimCl', 'hmimCl'].fix(0)
    m.fs.properties.PR_kappa_C['hmimCl', 'R125'].fix(0)
    m.fs.properties.PR_kappa_C['R125', 'R125'].fix(0)
    m.fs.properties.PR_kappa_C['R125', 'hmimCl'].fix(0)
    # Initialize the flash unit
    m.fs.state_block.initialize(outlvl=idaeslog.CRITICAL)

    # Fix the state variables on the state block
    m.fs.state_block.pressure.unfix()
    m.fs.state_block.mole_frac_comp['hmimCl'].unfix()
    m.fs.state_block.mole_frac_comp['R125'].unfix()
    m.fs.state_block.temperature.fix(float(data["temperature"]))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'R125'].fix(float(data['x_R125']))
    m.fs.state_block.mole_frac_phase_comp['Liq', 'hmimCl'].fix(float(data['x_hmimCl']))
    m.fs.state_block.mole_frac_comp['R125'].fix(float(data['x_R125'])+eps)
    m.fs.state_block.mole_frac_comp['hmimCl'].unfix()

    # Set bounds on variables to be estimated
    m.fs.properties.PR_kappa_A['hmimCl', 'R125'].setlb(-5)
    m.fs.properties.PR_kappa_A['hmimCl', 'R125'].setub(5)

    m.fs.properties.PR_kappa_A['R125', 'hmimCl'].setlb(-5)
    m.fs.properties.PR_kappa_A['R125', 'hmimCl'].setub(5)

    # Return initialized flash model
    return m

def SSE(m, data):
    """
    returns objective function expresion.

    Args:
        m: model
        data: pandas DataFrame with data

    Returns:
        objectuve function scaled expresion
    """
    expr = ((float(data["pressure"]) - m.fs.state_block.pressure)**2)

    return expr * 1e-6

variable_name = ["fs.properties.PR_kappa_A['hmimCl', 'R125']",
                 "fs.properties.PR_kappa_A['R125', 'hmimCl']"]


In [3]:
data = pd.read_csv('R125_hmimCl.csv')

print(data)

   temperature  pressure  x_R125  x_hmimCl
0       298.15     50000   0.040     0.960
1       298.15    100000   0.079     0.921
2       298.15    200000   0.160     0.840
3       298.15    400000   0.290     0.710
4       298.15    600000   0.414     0.586
5       298.15    800000   0.543     0.457
6       298.15   1000000   0.654     0.346


In [4]:
N = 100 #number of samples
results_params = np.zeros((N,2))
results_obj = np.zeros((N,1))

st_dev_T = 0.01 #C
st_dev_P = 9771 #MPa
st_dev_x = 0.10 #mole fraction

for i in range(N):
    print(i)
    noise_x = np.random.normal(loc=0,scale=st_dev_x)
    print(noise_x)
    data_ = data.copy()
    data_["x_hmimCl"] += data_["x_hmimCl"] * noise_x
    data_.loc[data_['x_hmimCl'] <= 0.0001, 'x_hmimCl'] = 0.0001
    data_.loc[data_['x_hmimCl'] >= 0.9999 , 'x_hmimCl'] = 0.9999      

    try:
        pest = parmest.Estimator(PR_model, data_, variable_name, SSE, tee=False)

        obj_value, parameters = pest.theta_est()

        for k,v in parameters.items():
            print(k, "=", v)

        line = 0

        for k,v in parameters.items():
            results_params[i,line] = v
            line += 1
        results_obj[i] = obj_value

    except ValueError:
        results_params[i,0] = 'NaN'
        results_params[i,1] = 'NaN'


0
-0.13297316939561651
2022-03-06 15:29:28 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:29:29 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:29:30 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:29:31 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:29:32 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:29:32 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:29:33 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[hmimCl,R125] = -0.07622286461872735
fs.properties.PR_kappa_A[R125,hmimCl] = -0.05747699310951432
1
0.0060

2022-03-06 15:30:31 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:30:32 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:30:33 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:30:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:30:36 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:30:37 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:30:38 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasibl

2022-03-06 15:31:43 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:31:44 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:31:45 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[hmimCl,R125] = -0.055204392173241475
fs.properties.PR_kappa_A[R125,hmimCl] = -0.04707576167535404
15
-0.012907572838643827
2022-03-06 15:31:48 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:31:49 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:31:50 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:31:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-

2022-03-06 15:32:57 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:32:58 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:32:59 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:33:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:33:01 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[hmimCl,R125] = -0.06319357538456152
fs.properties.PR_kappa_A[R125,hmimCl] = -0.048393694318987525
23
-0.04424596283758481
2022-03-06 15:33:05 [INFO] idaes.init.fs.state_block: Prope

2022-03-06 15:34:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:34:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:34:10 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
fs.properties.PR_kappa_A[hmimCl,R125] = -0.06293389298002683
fs.properties.PR_kappa_A[R125,hmimCl] = -0.04857156009135349
30
-0.19929055874570464
2022-03-06 15:34:13 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:34:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:34:15 [INFO] idaes.init.fs.state_block: Proper

...

2022-03-06 15:40:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[hmimCl,R125] = -0.0669664376189093
fs.properties.PR_kappa_A[R125,hmimCl] = -0.05267249410114225
67
-0.1273510036090141
2022-03-06 15:40:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:40:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:40:09 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:40:10 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:40:11 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:40:12 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-0

2022-03-06 15:41:14 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:41:15 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:41:16 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:41:17 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:41:18 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:41:19 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:41:20 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasibl

2022-03-06 15:42:31 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:42:34 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:42:35 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[hmimCl,R125] = -0.061107989567052524
fs.properties.PR_kappa_A[R125,hmimCl] = -0.04982082854472421
82
0.10649041345443522
2022-03-06 15:42:39 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:42:40 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:42:42 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:42:43 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03

2022-03-06 15:43:49 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:43:50 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:43:52 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:43:53 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:43:54 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:43:55 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:43:56 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
    model.name="_Q_opt";
      - termination condition: infeasible
      - message from solver: Ipopt 3.13.2\x3a Converged to a locally
        infeasibl

2022-03-06 15:45:00 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:45:01 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:45:02 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:45:03 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:45:04 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
fs.properties.PR_kappa_A[hmimCl,R125] = -0.06906141393289146
fs.properties.PR_kappa_A[R125,hmimCl] = -0.05372607199586149
97
-0.03155712933021585
2022-03-06 15:45:07 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03-06 15:45:08 [INFO] idaes.init.fs.state_block: Property package initialization: optimal - Optimal Solution Found.
2022-03

In [5]:
print(results_params)

[[-0.07622286 -0.05747699]
 [-0.0631421  -0.04819998]
 [-0.05935726 -0.04930459]
 [-0.06220559 -0.04880153]
 [-0.05828895 -0.04944659]
 [-0.06306545 -0.04850447]
 [-0.06706133 -0.05271981]
 [-0.06006197 -0.04921059]
 [-0.06319055 -0.04839702]
 [-0.05532647 -0.04713131]
 [-0.07656272 -0.05766114]
 [-0.06056929 -0.04914299]
 [-0.07951207 -0.05928446]
 [-0.07447373 -0.05653838]
 [-0.05520439 -0.04707576]
 [-0.05712373 -0.04795496]
 [-0.06388678 -0.05115667]
 [-0.05654708 -0.04768951]
 [-0.05597694 -0.04742816]
 [-0.07296985 -0.0557433 ]
 [-0.06286802 -0.04860136]
 [-0.06063512 -0.04913494]
 [-0.06319358 -0.04839369]
 [-0.06218793 -0.0503365 ]
 [-0.05375534 -0.04982693]
 [-0.06140621 -0.04899388]
 [-0.0596659  -0.04926347]
 [-0.07268501 -0.05559393]
 [-0.07142595 -0.05493817]
 [-0.06293389 -0.04857156]
 [-0.08640979 -0.06327156]
 [-0.06580576 -0.0520967 ]
 [-0.07874162 -0.05885596]
 [-0.07123019 -0.05483687]
 [-0.05881649 -0.04874081]
 [-0.06224152 -0.04879249]
 [-0.06318486 -0.04840299]
 

In [6]:
results = pd.DataFrame(results_params)
results.to_csv('R125_hmimCL_10_2.csv')